In [2]:
from numpy import dtype
import pandas as pd
from IPython.display import display
import matplotlib.pyplot as plt
from IPython.display import display
from PIL.EpsImagePlugin import split
import datetime
import time
import re

In [20]:
#Import tabelas de dados e prazos
idg_df = pd.read_excel('264.xlsx')
prazos_df = pd.read_excel('Prazos ISAP.xlsx')
idg_df


,SS,OS,Tipo,Especificação,Serviço,Unid Atual (OS),Equipe,Origem da SS,Localidade,Bairro,Logradouro,Ini do Serviço,Fim do Serviço,Encerramento,Mot de Encerr da OS,Sit da OS,Func. Encerrou OS,SS Atendida,Tipo de Operação,Data/Hora Última Tramitação da OS
0,969087481,1,26 - MANUTENCAO DE HIDROMETRO,4 - SUBSTITUICAO DE HIDROMETRO,12 - SUBSTITUICAO HID.,88880068 - CONSORCIO NOVA BOLANDEIRA,142-A - LOURIVAL NERI ANDRADE NETO,Interna,900 - SALVADOR UMB,52 - CAM DAS ARVORES,50110 - AA DOS SOMBREIROS,2022-06-08 15:04:00,2022-06-08 15:10:00,2022-06-08 15:12:33,0 - NAO INFORMADO,Concluída Executada,900032 - INTEGRACAO DIGITEAM,Sim,Comercial,2022-05-23 08:50:10
1,969087534,1,26 - MANUTENCAO DE HIDROMETRO,4 - SUBSTITUICAO DE HIDROMETRO,12 - SUBSTITUICAO HID.,88880068 - CONSORCIO NOVA BOLANDEIRA,142-A - LOURIVAL NERI ANDRADE NETO,Interna,900 - SALVADOR UMB,52 - CAM DAS ARVORES,50152 - AA DAS ESPATODIAS,NaT,NaT,2022-06-09 11:42:43,18 - SERVICO EXECUTADO P/ OUTRA OS,Concluída Não Executada,33620 - PATRICIA DA SILVA SANTOS,Não,Comercial,2022-05-23 08:50:09
2,969622865,2,26 - MANUTENCAO DE HIDROMETRO,4 - SUBSTITUICAO DE HIDROMETRO,12 - SUBSTITUICAO HID.,88880068 - CONSORCIO NOVA BOLANDEIRA,142-A - LOURIVAL NERI ANDRADE NETO,Interna,900 - SALVADOR UMB,52 - CAM DAS ARVORES,50225 - AA DOS MULUNGUS,NaT,NaT,2022-06-09 13:12:39,18 - SERVICO EXECUTADO P/ OUTRA OS,Concluída Não Executada,33620 - PATRICIA DA SILVA SANTOS,Não,Comercial,2022-04-08 14:12:57
3,970133832,2,26 - MANUTENCAO DE HIDROMETRO,19 - REVISAO HIDROMETRO PARADO,12 - SUBSTITUICAO HID.,88880068 - CONSORCIO NOVA BOLANDEIRA,150-A - ANDRE LUIZ CHAVES DOS SANTOS,Interna,900 - SALVADOR UMB,18 - ITAPUA,63700 - RU ITAPUA,NaT,NaT,2022-06-07 11:26:30,102 - LIGACAO ENCONTRADA CORTADA,Concluída Não Executada,33620 - PATRICIA DA SILVA SANTOS,Sim,Comercial,2022-05-09 07:59:34
4,970133842,2,26 - MANUTENCAO DE HIDROMETRO,19 - REVISAO HIDROMETRO PARADO,12 - SUBSTITUICAO HID.,88880068 - CONSORCIO NOVA BOLANDEIRA,143-A - YAN QUARESMA DOS SANTOS,Interna,900 - SALVADOR UMB,18 - ITAPUA,166495 - TV DAS PEDRINHAS DE ITAPUA 2A,2022-06-07 15:53:00,2022-06-07 16:03:00,2022-06-07 16:11:53,0 - NAO INFORMADO,Concluída Executada,900032 - INTEGRACAO DIGITEAM,Sim,Comercial,2022-05-09 07:59:35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1472,971668321,1,45 - VERIFICACAO DE FALTA DE AGUA,1 - FALTA DE AGUA NO IMOVEL,364 - VERIF FALTA AGUA IMOVEL,88880068 - CONSORCIO NOVA BOLANDEIRA,106-A - JOSEILTON SANTOS,Cliente,700 - LAURO DE FREITAS,10 - CENTRO,416439 - RU PATARO MACHADO,NaT,NaT,2022-06-09 14:26:27,20 - INVIABILIDADE ACESSO LOCAL,Concluída Não Executada,696849 - RONLISON CONCEICAO DA COSTA,Não,Água,2022-06-09 12:05:47
1473,971670422,1,26 - MANUTENCAO DE HIDROMETRO,2 - INSTALACAO DE HIDROMETRO,11 - INSTALACAO DE HIDROM.,88880068 - CONSORCIO NOVA BOLANDEIRA,NaN,Interna,900 - SALVADOR UMB,16 - BOCA DO RIO,96152 - RU PEDRO BATISTA DE ALMEIDA,2022-06-09 13:33:00,2022-06-09 13:36:00,2022-06-09 13:38:10,0 - NAO INFORMADO,Concluída Executada,202530 - JANETE DE SOUZA PEREIRA,Sim,Comercial,2022-06-09 13:32:36
1474,971671445,1,25 - MANUTENCAO DE ESGOTO,1 - EXTRAVASAMENTO DE ESGOTO RUA - PV,438 - DES RED ESG E PVS C/EQUIP,88880068 - CONSORCIO NOVA BOLANDEIRA,EQ325E - ALAN SANTOS TEXEIRA,Cliente,900 - SALVADOR UMB,16 - BOCA DO RIO,96012 - RU JOAO NUNES DA MATA,2022-06-09 14:42:00,2022-06-09 14:47:00,2022-06-09 15:03:37,0 - NAO INFORMADO,Concluída Executada,900032 - INTEGRACAO DIGITEAM,Sim,Esgoto,2022-06-09 13:57:31
1475,971671475,1,25 - MANUTENCAO DE ESGOTO,2 - EXTRAVASAMENTO DE ESGOTO RAMAL - CI,438 - DES RED ESG E PVS C/EQUIP,88880068 - CONSORCIO NOVA BOLANDEIRA,EQ325E - ALAN SANTOS TEXEIRA,Cliente,900 - SALVADOR UMB,16 - BOCA DO RIO,25174 - RU DO CAXUNDE,2022-06-09 14:55:00,2022-06-09 15:03:00,2022-06-09 15:04:55,0 - NAO INFORMADO,Concluída Executada,900032 - INTEGRACAO DIGITEAM,Sim,Esgoto,2022-06-09 13:58:05


In [21]:
#Alterar a localidade do bairro em especifico
idg_df.loc[idg_df['Bairro'] == '22 - JD MARGARIDAS', 'Localidade'] = '700 - LAURO DE FREITAS'
idg_df[['Bairro', 'Localidade']]



,Bairro,Localidade
0,52 - CAM DAS ARVORES,900 - SALVADOR UMB
1,52 - CAM DAS ARVORES,900 - SALVADOR UMB
2,52 - CAM DAS ARVORES,900 - SALVADOR UMB
3,18 - ITAPUA,900 - SALVADOR UMB
4,18 - ITAPUA,900 - SALVADOR UMB
...,...,...
1472,10 - CENTRO,700 - LAURO DE FREITAS
1473,16 - BOCA DO RIO,900 - SALVADOR UMB
1474,16 - BOCA DO RIO,900 - SALVADOR UMB
1475,16 - BOCA DO RIO,900 - SALVADOR UMB


In [22]:
#Criando coluna e concatenando de ID
idc1 = idg_df['IDC1'] = idg_df['Tipo'].map(lambda x : x.split(' - ')[0])
IDC2 = idg_df['IDC2'] = idg_df['Especificação'].map(lambda x : x.split(' - ')[0])

idg_df['ID_PRAZO'] = idg_df['IDC1'].map(str) + '.' + idg_df['IDC2'].map(str)
idg_df = idg_df.merge(prazos_df[['Item Contratual', 'Prazo Atendimento', 'ID_PRAZO']], on='ID_PRAZO')
idg_df

,SS,OS,Tipo,Especificação,Serviço,Unid Atual (OS),Equipe,Origem da SS,Localidade,Bairro,...,Sit da OS,Func. Encerrou OS,SS Atendida,Tipo de Operação,Data/Hora Última Tramitação da OS,IDC1,IDC2,ID_PRAZO,Item Contratual,Prazo Atendimento
0,969087481,1,26 - MANUTENCAO DE HIDROMETRO,4 - SUBSTITUICAO DE HIDROMETRO,12 - SUBSTITUICAO HID.,88880068 - CONSORCIO NOVA BOLANDEIRA,142-A - LOURIVAL NERI ANDRADE NETO,Interna,900 - SALVADOR UMB,52 - CAM DAS ARVORES,...,Concluída Executada,900032 - INTEGRACAO DIGITEAM,Sim,Comercial,2022-05-23 08:50:10,26,4,26.4,NAO IDENTIFICADO,5.0
1,969087534,1,26 - MANUTENCAO DE HIDROMETRO,4 - SUBSTITUICAO DE HIDROMETRO,12 - SUBSTITUICAO HID.,88880068 - CONSORCIO NOVA BOLANDEIRA,142-A - LOURIVAL NERI ANDRADE NETO,Interna,900 - SALVADOR UMB,52 - CAM DAS ARVORES,...,Concluída Não Executada,33620 - PATRICIA DA SILVA SANTOS,Não,Comercial,2022-05-23 08:50:09,26,4,26.4,NAO IDENTIFICADO,5.0
2,969622865,2,26 - MANUTENCAO DE HIDROMETRO,4 - SUBSTITUICAO DE HIDROMETRO,12 - SUBSTITUICAO HID.,88880068 - CONSORCIO NOVA BOLANDEIRA,142-A - LOURIVAL NERI ANDRADE NETO,Interna,900 - SALVADOR UMB,52 - CAM DAS ARVORES,...,Concluída Não Executada,33620 - PATRICIA DA SILVA SANTOS,Não,Comercial,2022-04-08 14:12:57,26,4,26.4,NAO IDENTIFICADO,5.0
3,969087480,2,26 - MANUTENCAO DE HIDROMETRO,4 - SUBSTITUICAO DE HIDROMETRO,12 - SUBSTITUICAO HID.,88880068 - CONSORCIO NOVA BOLANDEIRA,142-A - LOURIVAL NERI ANDRADE NETO,Interna,900 - SALVADOR UMB,52 - CAM DAS ARVORES,...,Concluída Não Executada,33620 - PATRICIA DA SILVA SANTOS,Não,Comercial,2022-05-01 11:25:23,26,4,26.4,NAO IDENTIFICADO,5.0
4,969087483,2,26 - MANUTENCAO DE HIDROMETRO,4 - SUBSTITUICAO DE HIDROMETRO,12 - SUBSTITUICAO HID.,88880068 - CONSORCIO NOVA BOLANDEIRA,142-A - LOURIVAL NERI ANDRADE NETO,Interna,900 - SALVADOR UMB,52 - CAM DAS ARVORES,...,Concluída Não Executada,33620 - PATRICIA DA SILVA SANTOS,Não,Comercial,2022-05-01 13:32:29,26,4,26.4,NAO IDENTIFICADO,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1472,971598484,1,25 - MANUTENCAO DE ESGOTO,3 - LIMPEZA GERAL,578 - RETIRADA EXPURGO ESGOTO,88880068 - CONSORCIO NOVA BOLANDEIRA,309E - JOSE MOREIRA OLIVEIRA,Interna,900 - SALVADOR UMB,15 - PITUBA,...,Concluída Executada,900032 - INTEGRACAO DIGITEAM,Sim,Esgoto,2022-06-07 16:57:40,25,3,25.3,ESGOTO,1.0
1473,971662182,1,25 - MANUTENCAO DE ESGOTO,3 - LIMPEZA GERAL,578 - RETIRADA EXPURGO ESGOTO,88880068 - CONSORCIO NOVA BOLANDEIRA,309E - JOSE MOREIRA OLIVEIRA,Interna,900 - SALVADOR UMB,141 - PITUACU,...,Concluída Executada,900032 - INTEGRACAO DIGITEAM,Sim,Esgoto,2022-06-09 10:13:39,25,3,25.3,ESGOTO,1.0
1474,971626868,1,27 - MANUTENCAO DE RAMAL DE AGUA,4 - TRANSFERENCIA DE RAMAL,300 - TRANS RAM E/OU HIDRO 1/2,88880068 - CONSORCIO NOVA BOLANDEIRA,306-A - PAULO SERGIO BISPO PEREIRA,Interna,900 - SALVADOR UMB,18 - ITAPUA,...,Concluída Executada,203553 - DIEGO DE OLIVEIRA FONSECA,Sim,Comercial,2022-06-08 11:24:16,27,4,27.4,AGUA,3.0
1475,971627003,1,27 - MANUTENCAO DE RAMAL DE AGUA,4 - TRANSFERENCIA DE RAMAL,300 - TRANS RAM E/OU HIDRO 1/2,88880068 - CONSORCIO NOVA BOLANDEIRA,306-A - PAULO SERGIO BISPO PEREIRA,Interna,900 - SALVADOR UMB,18 - ITAPUA,...,Concluída Executada,203553 - DIEGO DE OLIVEIRA FONSECA,Sim,Comercial,2022-06-08 11:26:53,27,4,27.4,AGUA,3.0


In [23]:
#idg_df['Data/Hora Última Tramitação da OS'] = pd.to_datetime(idg_df['Data/Hora Última Tramitação da OS'],format="%d/%m/%Y, %H:%M:%S")
#idg_df['Encerramento'] = pd.to_datetime(idg_df['Encerramento'],format="%d/%m/%Y, %H:%M:%S")
idg_df['Encerramento']

0      2022-06-08 15:12:33
1      2022-06-09 11:42:43
2      2022-06-09 13:12:39
3      2022-06-09 13:11:12
4      2022-06-09 11:43:56
               ...        
1472   2022-06-08 12:10:05
1473   2022-06-09 11:03:55
1474   2022-06-08 13:48:09
1475   2022-06-08 13:48:38
1476   2022-06-09 13:38:10
Name: Encerramento, Length: 1477, dtype: datetime64[ns]

In [24]:
#criando colunas Data e Hora Separas
idg_df['Data']      = idg_df['Data/Hora Última Tramitação da OS'].dt.strftime('%d-%m-%Y')
idg_df['Hora_Tram'] = idg_df['Data/Hora Última Tramitação da OS'].dt.strftime('%H:%M:%S')
idg_df['Hora_Ence'] = idg_df['Encerramento'].dt.strftime('%H:%M:%S')
#.dtypes Conluta o tipo de coluna
idg_df[['Data', 'Hora_Tram', 'Hora_Ence']]

,Data,Hora_Tram,Hora_Ence
0,23-05-2022,08:50:10,15:12:33
1,23-05-2022,08:50:09,11:42:43
2,08-04-2022,14:12:57,13:12:39
3,01-05-2022,11:25:23,13:11:12
4,01-05-2022,13:32:29,11:43:56
...,...,...,...
1472,07-06-2022,16:57:40,12:10:05
1473,09-06-2022,10:13:39,11:03:55
1474,08-06-2022,11:24:16,13:48:09
1475,08-06-2022,11:26:53,13:48:38


In [25]:
#Criando coluna 'Tempo de Atendimento'
idg_df['Tempo de Atendimento'] = idg_df['Encerramento'] - idg_df['Data/Hora Última Tramitação da OS']
idg_df

,SS,OS,Tipo,Especificação,Serviço,Unid Atual (OS),Equipe,Origem da SS,Localidade,Bairro,...,Data/Hora Última Tramitação da OS,IDC1,IDC2,ID_PRAZO,Item Contratual,Prazo Atendimento,Data,Hora_Tram,Hora_Ence,Tempo de Atendimento
0,969087481,1,26 - MANUTENCAO DE HIDROMETRO,4 - SUBSTITUICAO DE HIDROMETRO,12 - SUBSTITUICAO HID.,88880068 - CONSORCIO NOVA BOLANDEIRA,142-A - LOURIVAL NERI ANDRADE NETO,Interna,900 - SALVADOR UMB,52 - CAM DAS ARVORES,...,2022-05-23 08:50:10,26,4,26.4,NAO IDENTIFICADO,5.0,23-05-2022,08:50:10,15:12:33,16 days 06:22:23
1,969087534,1,26 - MANUTENCAO DE HIDROMETRO,4 - SUBSTITUICAO DE HIDROMETRO,12 - SUBSTITUICAO HID.,88880068 - CONSORCIO NOVA BOLANDEIRA,142-A - LOURIVAL NERI ANDRADE NETO,Interna,900 - SALVADOR UMB,52 - CAM DAS ARVORES,...,2022-05-23 08:50:09,26,4,26.4,NAO IDENTIFICADO,5.0,23-05-2022,08:50:09,11:42:43,17 days 02:52:34
2,969622865,2,26 - MANUTENCAO DE HIDROMETRO,4 - SUBSTITUICAO DE HIDROMETRO,12 - SUBSTITUICAO HID.,88880068 - CONSORCIO NOVA BOLANDEIRA,142-A - LOURIVAL NERI ANDRADE NETO,Interna,900 - SALVADOR UMB,52 - CAM DAS ARVORES,...,2022-04-08 14:12:57,26,4,26.4,NAO IDENTIFICADO,5.0,08-04-2022,14:12:57,13:12:39,61 days 22:59:42
3,969087480,2,26 - MANUTENCAO DE HIDROMETRO,4 - SUBSTITUICAO DE HIDROMETRO,12 - SUBSTITUICAO HID.,88880068 - CONSORCIO NOVA BOLANDEIRA,142-A - LOURIVAL NERI ANDRADE NETO,Interna,900 - SALVADOR UMB,52 - CAM DAS ARVORES,...,2022-05-01 11:25:23,26,4,26.4,NAO IDENTIFICADO,5.0,01-05-2022,11:25:23,13:11:12,39 days 01:45:49
4,969087483,2,26 - MANUTENCAO DE HIDROMETRO,4 - SUBSTITUICAO DE HIDROMETRO,12 - SUBSTITUICAO HID.,88880068 - CONSORCIO NOVA BOLANDEIRA,142-A - LOURIVAL NERI ANDRADE NETO,Interna,900 - SALVADOR UMB,52 - CAM DAS ARVORES,...,2022-05-01 13:32:29,26,4,26.4,NAO IDENTIFICADO,5.0,01-05-2022,13:32:29,11:43:56,38 days 22:11:27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1472,971598484,1,25 - MANUTENCAO DE ESGOTO,3 - LIMPEZA GERAL,578 - RETIRADA EXPURGO ESGOTO,88880068 - CONSORCIO NOVA BOLANDEIRA,309E - JOSE MOREIRA OLIVEIRA,Interna,900 - SALVADOR UMB,15 - PITUBA,...,2022-06-07 16:57:40,25,3,25.3,ESGOTO,1.0,07-06-2022,16:57:40,12:10:05,0 days 19:12:25
1473,971662182,1,25 - MANUTENCAO DE ESGOTO,3 - LIMPEZA GERAL,578 - RETIRADA EXPURGO ESGOTO,88880068 - CONSORCIO NOVA BOLANDEIRA,309E - JOSE MOREIRA OLIVEIRA,Interna,900 - SALVADOR UMB,141 - PITUACU,...,2022-06-09 10:13:39,25,3,25.3,ESGOTO,1.0,09-06-2022,10:13:39,11:03:55,0 days 00:50:16
1474,971626868,1,27 - MANUTENCAO DE RAMAL DE AGUA,4 - TRANSFERENCIA DE RAMAL,300 - TRANS RAM E/OU HIDRO 1/2,88880068 - CONSORCIO NOVA BOLANDEIRA,306-A - PAULO SERGIO BISPO PEREIRA,Interna,900 - SALVADOR UMB,18 - ITAPUA,...,2022-06-08 11:24:16,27,4,27.4,AGUA,3.0,08-06-2022,11:24:16,13:48:09,0 days 02:23:53
1475,971627003,1,27 - MANUTENCAO DE RAMAL DE AGUA,4 - TRANSFERENCIA DE RAMAL,300 - TRANS RAM E/OU HIDRO 1/2,88880068 - CONSORCIO NOVA BOLANDEIRA,306-A - PAULO SERGIO BISPO PEREIRA,Interna,900 - SALVADOR UMB,18 - ITAPUA,...,2022-06-08 11:26:53,27,4,27.4,AGUA,3.0,08-06-2022,11:26:53,13:48:38,0 days 02:21:45


In [11]:
idg_df['Tempo de Atendimento']


0      16 days 06:22:23
1      17 days 02:52:34
2      61 days 22:59:42
3      39 days 01:45:49
4      38 days 22:11:27
             ...       
1472    0 days 19:12:25
1473    0 days 00:50:16
1474    0 days 02:23:53
1475    0 days 02:21:45
1476    0 days 00:05:34
Name: Tempo de Atendimento, Length: 1477, dtype: timedelta64[ns]

In [12]:
idg_df['Tempo de Atendimento'] = idg_df['Tempo de Atendimento'].astype('float')
max(idg_df['Tempo de Atendimento'])

TypeError: Cannot cast TimedeltaArray to dtype float64

In [13]:
idg_df['Prazo Atendimento'] = idg_df['Prazo Atendimento'].astype('float')
idg_df['Prazo Atendimento'] 

0       5.0
1       5.0
2       5.0
3       5.0
4       5.0
       ... 
1472    1.0
1473    1.0
1474    3.0
1475    3.0
1476    0.0
Name: Prazo Atendimento, Length: 1477, dtype: float64

In [14]:
lista_a = idg_df['Tempo de Atendimento']
lista_b = idg_df['Prazo Atendimento']

In [15]:
lista_a = idg_df['Tempo de Atendimento']
lista_b = idg_df['Prazo Atendimento']

for index, data in enumerate(zip(lista_a,lista_b)):
    i, j = data
    if i < j:
        print("oi")


TypeError: '<' not supported between instances of 'Timedelta' and 'float'

In [18]:
#Coluna de majoração
idg_df['Prazo Atendimento'] = idg_df['Prazo Atendimento'].astype('int64')
idg_df['Tempo de Atendimento'] = idg_df['Tempo de Atendimento'].astype('int64')

idg_df['Majoração'] = idg_df['Tempo de Atendimento'] / idg_df['Prazo Atendimento'].round()
idg_df['Majoração'] = idg_df['Majoração'].astype('float')
idg_df['Majoração'] = idg_df['Majoração'].round()
idg_df[['Tempo de Atendimento', 'Prazo Atendimento', 'Majoração']]

,Tempo de Atendimento,Prazo Atendimento,Majoração
0,1405343000000000,5,2.810686e+14
1,1479154000000000,5,2.958308e+14
2,5353182000000000,5,1.070636e+15
3,3375949000000000,5,6.751898e+14
4,3363087000000000,5,6.726174e+14
...,...,...,...
1472,69145000000000,1,6.914500e+13
1473,3016000000000,1,3.016000e+12
1474,8633000000000,3,2.877667e+12
1475,8505000000000,3,2.835000e+12


In [19]:
idg_df

,SS,OS,Tipo,Especificação,Serviço,Unid Atual (OS),Equipe,Origem da SS,Localidade,Bairro,...,IDC1,IDC2,ID_PRAZO,Item Contratual,Prazo Atendimento,Data,Hora_Tram,Hora_Ence,Tempo de Atendimento,Majoração
0,969087481,1,26 - MANUTENCAO DE HIDROMETRO,4 - SUBSTITUICAO DE HIDROMETRO,12 - SUBSTITUICAO HID.,88880068 - CONSORCIO NOVA BOLANDEIRA,142-A - LOURIVAL NERI ANDRADE NETO,Interna,900 - SALVADOR UMB,52 - CAM DAS ARVORES,...,26,4,26.4,NAO IDENTIFICADO,5,23-05-2022,08:50:10,15:12:33,1405343000000000,2.810686e+14
1,969087534,1,26 - MANUTENCAO DE HIDROMETRO,4 - SUBSTITUICAO DE HIDROMETRO,12 - SUBSTITUICAO HID.,88880068 - CONSORCIO NOVA BOLANDEIRA,142-A - LOURIVAL NERI ANDRADE NETO,Interna,900 - SALVADOR UMB,52 - CAM DAS ARVORES,...,26,4,26.4,NAO IDENTIFICADO,5,23-05-2022,08:50:09,11:42:43,1479154000000000,2.958308e+14
2,969622865,2,26 - MANUTENCAO DE HIDROMETRO,4 - SUBSTITUICAO DE HIDROMETRO,12 - SUBSTITUICAO HID.,88880068 - CONSORCIO NOVA BOLANDEIRA,142-A - LOURIVAL NERI ANDRADE NETO,Interna,900 - SALVADOR UMB,52 - CAM DAS ARVORES,...,26,4,26.4,NAO IDENTIFICADO,5,08-04-2022,14:12:57,13:12:39,5353182000000000,1.070636e+15
3,969087480,2,26 - MANUTENCAO DE HIDROMETRO,4 - SUBSTITUICAO DE HIDROMETRO,12 - SUBSTITUICAO HID.,88880068 - CONSORCIO NOVA BOLANDEIRA,142-A - LOURIVAL NERI ANDRADE NETO,Interna,900 - SALVADOR UMB,52 - CAM DAS ARVORES,...,26,4,26.4,NAO IDENTIFICADO,5,01-05-2022,11:25:23,13:11:12,3375949000000000,6.751898e+14
4,969087483,2,26 - MANUTENCAO DE HIDROMETRO,4 - SUBSTITUICAO DE HIDROMETRO,12 - SUBSTITUICAO HID.,88880068 - CONSORCIO NOVA BOLANDEIRA,142-A - LOURIVAL NERI ANDRADE NETO,Interna,900 - SALVADOR UMB,52 - CAM DAS ARVORES,...,26,4,26.4,NAO IDENTIFICADO,5,01-05-2022,13:32:29,11:43:56,3363087000000000,6.726174e+14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1472,971598484,1,25 - MANUTENCAO DE ESGOTO,3 - LIMPEZA GERAL,578 - RETIRADA EXPURGO ESGOTO,88880068 - CONSORCIO NOVA BOLANDEIRA,309E - JOSE MOREIRA OLIVEIRA,Interna,900 - SALVADOR UMB,15 - PITUBA,...,25,3,25.3,ESGOTO,1,07-06-2022,16:57:40,12:10:05,69145000000000,6.914500e+13
1473,971662182,1,25 - MANUTENCAO DE ESGOTO,3 - LIMPEZA GERAL,578 - RETIRADA EXPURGO ESGOTO,88880068 - CONSORCIO NOVA BOLANDEIRA,309E - JOSE MOREIRA OLIVEIRA,Interna,900 - SALVADOR UMB,141 - PITUACU,...,25,3,25.3,ESGOTO,1,09-06-2022,10:13:39,11:03:55,3016000000000,3.016000e+12
1474,971626868,1,27 - MANUTENCAO DE RAMAL DE AGUA,4 - TRANSFERENCIA DE RAMAL,300 - TRANS RAM E/OU HIDRO 1/2,88880068 - CONSORCIO NOVA BOLANDEIRA,306-A - PAULO SERGIO BISPO PEREIRA,Interna,900 - SALVADOR UMB,18 - ITAPUA,...,27,4,27.4,AGUA,3,08-06-2022,11:24:16,13:48:09,8633000000000,2.877667e+12
1475,971627003,1,27 - MANUTENCAO DE RAMAL DE AGUA,4 - TRANSFERENCIA DE RAMAL,300 - TRANS RAM E/OU HIDRO 1/2,88880068 - CONSORCIO NOVA BOLANDEIRA,306-A - PAULO SERGIO BISPO PEREIRA,Interna,900 - SALVADOR UMB,18 - ITAPUA,...,27,4,27.4,AGUA,3,08-06-2022,11:26:53,13:48:38,8505000000000,2.835000e+12
